# Crime Articles Recommendation System

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [29]:
df = pd.read_csv(r"C:\Users\athar\Desktop\vs code projecs\projects\news articles recommendation system\Crime_Articles.csv")

## Basic EDA

In [3]:
df.shape

(81096, 8)

In [4]:
df.head()

,heading,content_summary,article_link,img_link,month,date,time,Year
0,2 society staffers thrash resident in Greater...,Two maintenance staffers of a highrise reside...,https://timesofindia.indiatimes.com/city/noida...,https://static.toiimg.com/thumb/imgsize-123456...,Oct,14,06:37,2022
1,2 society staffers thrash resident in Greater...,Two maintenance staffers of a highrise reside...,https://timesofindia.indiatimes.com/city/noida...,https://static.toiimg.com/thumb/imgsize-123456...,Oct,14,06:37,2022
2,Delhi: 2 steal jewels worth Rs 7 lakh from te...,A man and a woman allegedly stole jewels wort...,https://timesofindia.indiatimes.com/city/delhi...,https://static.toiimg.com/thumb/imgsize-123456...,Oct,12,11:44,2022
3,2 society staffers thrash resident in Greater...,Two maintenance staffers of a highrise reside...,https://timesofindia.indiatimes.com/city/noida...,https://static.toiimg.com/thumb/imgsize-123456...,Oct,14,06:37,2022
4,Delhi: 2 steal jewels worth Rs 7 lakh from te...,A man and a woman allegedly stole jewels wort...,https://timesofindia.indiatimes.com/city/delhi...,https://static.toiimg.com/thumb/imgsize-123456...,Oct,12,11:44,2022


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81096 entries, 0 to 81095
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   heading          81096 non-null  object
 1   content_summary  81096 non-null  object
 2   article_link     81096 non-null  object
 3   img_link         81096 non-null  object
 4   month            81096 non-null  object
 5   date             81096 non-null  int64 
 6   time             81096 non-null  object
 7   Year             81096 non-null  object
dtypes: int64(1), object(7)
memory usage: 4.9+ MB


In [25]:
df.isna().sum()

heading            0
content_summary    0
article_link       0
img_link           0
month              0
date               0
time               0
Year               0
dtype: int64

In [26]:
df.columns

Index(['heading', 'content_summary', 'article_link', 'img_link', 'month',
       'date', 'time', 'Year'],
      dtype='object')

### Only the "heading" and "content_summary" columns seem to be useful for generating tags

In [51]:
selected_features = df[["heading", "content_summary"]]

In [52]:
selected_features.head()

,heading,content_summary
0,2 society staffers thrash resident in Greater...,Two maintenance staffers of a highrise reside...
1,2 society staffers thrash resident in Greater...,Two maintenance staffers of a highrise reside...
2,Delhi: 2 steal jewels worth Rs 7 lakh from te...,A man and a woman allegedly stole jewels wort...
3,2 society staffers thrash resident in Greater...,Two maintenance staffers of a highrise reside...
4,Delhi: 2 steal jewels worth Rs 7 lakh from te...,A man and a woman allegedly stole jewels wort...


In [53]:
selected_features.shape

(81096, 2)

### Removing the duplicate articles

In [54]:
selected_features = selected_features.drop_duplicates()

In [55]:
selected_features.shape

(6586, 2)

In [56]:
selected_features = selected_features.reset_index()

In [57]:
selected_features.drop("index", inplace=True, axis=1)

In [58]:
selected_features.head()

,heading,content_summary
0,2 society staffers thrash resident in Greater...,Two maintenance staffers of a highrise reside...
1,Delhi: 2 steal jewels worth Rs 7 lakh from te...,A man and a woman allegedly stole jewels wort...
2,Chennai: Three held for performing stunts in ...,A day after seven youth performed stunts in a...
3,Man held for stabbing wife in Chennai,Ayanavaram police arrested a 20-year-old man ...
4,Madhya Pradesh man lays electric wire to kill...,A man in Madhya Pradesh's Betul district laid...


## Text Preprocessing

In [64]:
def strip(text : str):
    text = text.lstrip()
    text = text.rstrip()
    return text

In [65]:
selected_features["heading"] = selected_features["heading"].apply(strip)
selected_features["content_summary"] = selected_features["content_summary"].apply(strip)

In [67]:
selected_features["tags"] = selected_features["heading"] + " " + selected_features["content_summary"]

In [68]:
selected_features.head()

,heading,content_summary,tags
0,2 society staffers thrash resident in Greater ...,Two maintenance staffers of a highrise residen...,2 society staffers thrash resident in Greater ...
1,Delhi: 2 steal jewels worth Rs 7 lakh from tem...,A man and a woman allegedly stole jewels worth...,Delhi: 2 steal jewels worth Rs 7 lakh from tem...
2,Chennai: Three held for performing stunts in t...,A day after seven youth performed stunts in a ...,Chennai: Three held for performing stunts in t...
3,Man held for stabbing wife in Chennai,Ayanavaram police arrested a 20-year-old man o...,Man held for stabbing wife in Chennai Ayanavar...
4,Madhya Pradesh man lays electric wire to kill ...,A man in Madhya Pradesh's Betul district laid ...,Madhya Pradesh man lays electric wire to kill ...


In [77]:
import string

In [79]:
def fun(text : str):
    text = text.translate(str.maketrans('','',string.punctuation))
    return text

In [80]:
selected_features["tags"] = selected_features["tags"].apply(fun)

In [82]:
selected_features["tags"][0]

'2 society staffers thrash resident in Greater Noida held Two maintenance staffers of a highrise residential society in Greater Noida West have been arrested for allegedly thrashing a 59yearold resident with sticks on Wednesday night'

In [84]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [85]:
stemmer = PorterStemmer()

In [93]:
def fun(text : str):
    temp = ""
    words = word_tokenize(text)
    for word in words:
        temp += stemmer.stem(word) + " "
    return temp.rstrip()

In [89]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\athar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [95]:
selected_features["tags"] = selected_features["tags"].apply(fun)

In [96]:
selected_features.head()

,heading,content_summary,tags
0,2 society staffers thrash resident in Greater ...,Two maintenance staffers of a highrise residen...,2 societi staffer thrash resid in greater noid...
1,Delhi: 2 steal jewels worth Rs 7 lakh from tem...,A man and a woman allegedly stole jewels worth...,delhi 2 steal jewel worth rs 7 lakh from templ...
2,Chennai: Three held for performing stunts in t...,A day after seven youth performed stunts in a ...,chennai three held for perform stunt in train ...
3,Man held for stabbing wife in Chennai,Ayanavaram police arrested a 20-year-old man o...,man held for stab wife in chennai ayanavaram p...
4,Madhya Pradesh man lays electric wire to kill ...,A man in Madhya Pradesh's Betul district laid ...,madhya pradesh man lay electr wire to kill wif...


## Vectorizing the tags

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [99]:
vectorizer = TfidfVectorizer(stop_words="english")

In [109]:
vectors = vectorizer.fit_transform(selected_features["tags"]).toarray()

In [110]:
vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [111]:
vectorizer.get_feature_names()

['0000',
 '10',
 '100',
 '1000',
 '10000',
 '1000kg',
 '100130',
 '1002',
 '100200',
 '100kg',
 '100km',
 '100m',
 '100plu',
 '101',
 '1010pm',
 '1012',
 '1015',
 '1019',
 '101st',
 '102',
 '1030am',
 '1030pm',
 '10349',
 '103rd',
 '104',
 '1040am',
 '104crore',
 '105',
 '1050',
 '10500',
 '105000',
 '1055pm',
 '106',
 '1060',
 '1075',
 '108',
 '109',
 '1098',
 '10a',
 '10am',
 '10day',
 '10ftx10ft',
 '10k',
 '10kg',
 '10km',
 '10monthold',
 '10pm',
 '10year',
 '10yearold',
 '10yrold',
 '11',
 '110',
 '1100',
 '11000',
 '11000r',
 '1106',
 '11087',
 '110g',
 '111',
 '1113',
 '111kg',
 '112',
 '113',
 '1130',
 '1135',
 '114',
 '1142',
 '1149',
 '115',
 '1154',
 '115pm',
 '116',
 '1165kg',
 '117',
 '1176',
 '118',
 '119',
 '11accus',
 '11am',
 '11kg',
 '11monthold',
 '11pm',
 '11yearold',
 '11yrold',
 '12',
 '120',
 '1200',
 '12000',
 '120000',
 '1200crore',
 '120b',
 '121',
 '1225',
 '123',
 '1230',
 '1230am',
 '1230pm',
 '124',
 '1245',
 '1246',
 '125',
 '1250',
 '12537',
 '125kg',
 '1

## Calculating cosine similarity between the articles

In [112]:
from sklearn.metrics.pairwise import cosine_similarity

In [113]:
similarity = cosine_similarity(vectors)

In [114]:
similarity

array([[1.        , 0.00533196, 0.01232584, ..., 0.        , 0.        ,
        0.        ],
       [0.00533196, 1.        , 0.03812153, ..., 0.02115182, 0.00634171,
        0.03425904],
       [0.01232584, 0.03812153, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.02115182, 0.        , ..., 1.        , 0.00769421,
        0.00303502],
       [0.        , 0.00634171, 0.        , ..., 0.00769421, 1.        ,
        0.05659097],
       [0.        , 0.03425904, 0.        , ..., 0.00303502, 0.05659097,
        1.        ]])

In [115]:
similarity.shape

(6586, 6586)

In [116]:
similarity[0]

array([1.        , 0.00533196, 0.01232584, ..., 0.        , 0.        ,
       0.        ])

## Building the recommendation system

In [120]:
def recommend(article_heading):
  # getting the index of the article in the dataframe
  article_index = selected_features[selected_features["heading"] == article_heading].index[0]

  # getting the distance vector for the article
  distances = similarity[article_index]

  # enumerating the distances so that the indices are not lost
  # sorting the list according to the distances and keeping the top 5 articles
  articles_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x : x[1])[1:6]

  # printing the list of the top 5 similar articles
  for article in articles_list:
    print(selected_features.iloc[article[0]]["heading"])

In [137]:
recommend("Delhi: 2 steal jewels worth Rs 7 lakh from temple in Pitampura, caught on CCTV")

Man poses as jeweller to get hands on Rs 21 lakh gold in Mumbai, arrested
Thane: Robber loots jeweller of gold worth Rs 4 lakh
Woman held for duping jeweller in Mumbai
Mumbai: Two held for stealing donation box containing Rs 2 lakh cash from temple
Four rob Rs 40 lakh booty from jeweller's staff in Mumbai
